2. Pipeline: All the transformation following the CRISP-DM methodology.

In [ ]:
# if you are using a google drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Berkeley/Unit11/practical_application_II')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer

# Because I got Session Crashed after using all available RAM
# I decided to drop the column VIN ( I don't think the VIN number has any relevant importance predicting the price )
# and used dtype_dict
dtype_dict = {
    'price': 'int32',
    'year': 'float32',
    'odometer': 'float32'
}
cars = pd.read_csv('data/vehicles.csv', dtype=dtype_dict)
cars.drop(columns=['VIN'], inplace=True)

# and take  a random 50% of the total rows of the dataset
cars_sample = cars.sample(frac=0.5, random_state=42)

numerical_cols = ['price', 'year', 'odometer']  # Before log transformation
# Build a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        # I could add more and more transformers for categorical variables here as needed,but no time :-)
    ],
    remainder='passthrough'
)


# ========= Step 1: Outlier Detection and Removal =========
def remove_outliers_iqr(df, numeric_cols=['price', 'year', 'odometer'], factor=1.5):
    """
    Removing those rows from the numeroc colums with outlier values using the IQR method.

    Parameters:
      df (pd.DataFrame): Input DataFrame.
      numeric_cols (list): List of numeric columns to check.
      factor (float): Multiplier for the IQR to define outlier thresholds.

    Returns:
      pd.DataFrame: DataFrame with outliers removed.
    """
    df = df.copy()
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - factor * IQR
        upper_bound = Q3 + factor * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# ========= Step 2: Missing Value Imputation =========
def impute_missing_values(df):
    df = df.copy()
    # For numeric columns, fill missing values with the median
    numeric_cols = ['year', 'odometer', 'price']
    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].median())

    # For categorical columns, fill missing values with 'Unknown'
    # Not considering state .. I don't think the state could make any difference
    categorical_cols = ['region', 'manufacturer', 'model', 'condition',
                        'cylinders', 'fuel', 'title_status', 'transmission',
                        'drive', 'size', 'type', 'paint_color']
    for col in categorical_cols:
        df[col] = df[col].fillna('Unknown')
        df['year'] = df['year'].astype('int32')

    return df

# ========= Step 3: Outlier Treatment and Odometer Transformation =========
def process_odometer(df):
    df = df.copy()
    # Cap the odometer values at the 1st and 99th percentiles
    lower_bound = df['odometer'].quantile(0.01)
    upper_bound = df['odometer'].quantile(0.99)
    df['odometer_capped'] = df['odometer'].clip(lower=lower_bound, upper=upper_bound)

    # Log-transform the capped odometer to reduce right skewness (use log1p to handle zero values)
    df['odometer_log'] = np.log1p(df['odometer_capped'])
    return df

# ========= Step 4: Feature Engineering =========
def feature_engineering(df):
    df = df.copy()
    # Create a new feature: car age (current year - production year)
    current_year = datetime.datetime.now().year
    df['age'] = current_year - df['year'].astype(int)

    # Create price per mile; avoid division by zero by replacing zero with NaN first
    # instead of remove the whole row to preserv potential valid data. Like new vehicles.
    # can introduce some bias in my analysis...
    # Create price per mile; handle potential division by zero by filling with NaN first
    df['price_per_mile'] = df['price'] / df['odometer']
    # Replace inf with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Fill NaN values with median of price_per_mile column
    df['price_per_mile'].fillna(df['price_per_mile'].median(), inplace=True)

    return df

# ========= Step 5: Reduce Categories for Manufacturer =========
def reduce_manufacturer(df, threshold=0.01):
  # I tested several values for the threshold starting with 0.05, I realized 0.01 gives me fair list of
  # manufacturers. O.05 only gaveme Toyota Chevrolet and Ford
    df = df.copy()
    # Calculate the relative frequency of each manufacturer
    freq = df['manufacturer'].value_counts(normalize=True)
    # Identify manufacturers that represent at least 'threshold' of observations
    frequent = freq[freq >= threshold].index
    # Create a new column where infrequent manufacturers are replaced with 'Other'
    df['manufacturer_reduced'] = df['manufacturer'].apply(
        lambda x: x if x in frequent else 'Other'
    )
    return df

# ========= Step 6: Ordinal Encoding =========
def ordinal_encoding(df):
    df = df.copy()
    # Based in previous analysis These are the ordinal mappings..
    condition_mapping = {
        'salvage': 1,
        'fair': 2,
        'good': 3,
        'excellent': 4,
        'like new': 5,
        'new': 6,
        'Unknown': np.nan
    }
    title_status_mapping = {
        'salvage': 1,
        'rebuilt': 2,
        'clean': 3,
        'Unknown': np.nan
    }
    size_mapping = {
        'compact': 1,
        'midsize': 2,
        'fullsize': 3,
        'Unknown': np.nan
    }

    df['condition_ordinal'] = df['condition'].map(condition_mapping)
    df['title_status_ordinal'] = df['title_status'].map(title_status_mapping)
    df['size_ordinal'] = df['size'].map(size_mapping)

    return df

# ========= Step 7: One-Hot Encoding for Remaining Categorical Variables =========
def one_hot_encoding(df):
    df = df.copy()
    # List the remaining categorical columns to one-hot encode.
    # Exclude columns that have already been ordinal-encoded or transformed.
    cols_to_encode = ['manufacturer', 'cylinders', 'fuel',
                      'transmission']
    df = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)
    return df

# ========= Full Preprocessing Function =========
def full_preprocessing(df):

    # Step 0: Apply ColumnTransformer for scaling (NEW STEP)
    df_transformed = preprocessor.fit_transform(df) # return a numpy array
    # ----> Convert back to DataFrame (important!)
    remainder_columns = [col for col in df.columns if col not in numerical_cols]
    all_columns = numerical_cols + remainder_columns
    df = pd.DataFrame(df_transformed, columns=all_columns)

    df = pd.DataFrame(df, columns=numerical_cols + list(df.columns[len(numerical_cols):]))
    # Step 1: Remove outliers using the IQR method
    df = remove_outliers_iqr(df, numeric_cols = numerical_cols, factor=1.5)
    # Step 2: Impute missing values
    df = impute_missing_values(df)
    # Step 3: Process the odometer variable (cap and log-transform)
    df = process_odometer(df)
    # Step 4: Create additional features (age, price per mile)
    df = feature_engineering(df)
    # Step 5: Reduce manufacturer categories
    df = reduce_manufacturer(df)
    # Step 6: Apply ordinal encoding on select columns
    df = ordinal_encoding(df)
    # Step 7: One-hot encode the remaining categorical variables
    df = one_hot_encoding(df)
    return df

# ========= Create the Pipeline =========
preprocessing_pipeline = Pipeline(steps=[
    ('full_preprocessing', FunctionTransformer(full_preprocessing))
])

# ========= Apply the Pipeline =========
# need to load the sample 50% to avoid memory RAM exhausting
# cars = pd.read_csv('data/vehicles.csv')

# Run the full preprocessing pipeline with the 50% of the full date set to avoid memory issues
cars_preprocessed = preprocessing_pipeline.transform(cars_sample)

# Display the first few rows of the preprocessed data
print(cars_preprocessed.head())



<ipython-input-16-939525ec2d46>:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].median())
<ipython-input-16-939525ec2d46>:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].median())
<ipython-input-16-939525ec2d46>:62: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] 

      price  year  odometer          id                  region  \
0  0.003609     0 -0.270731  7315883828                lakeland   
3 -0.000587     0 -0.024469  7312663807      northern panhandle   
4 -0.003020     0 -0.230290  7315368523                  eugene   
5 -0.002642     0  0.586199  7309863303  waterloo / cedar falls   
6 -0.000351     0 -0.277230  7315163492                 jackson   

                   model  condition title_status    drive     size  ...  \
0  f150 super cab lariat       good        clean      4wd  Unknown  ...   
3                   328i    Unknown        clean  Unknown  Unknown  ...   
4            suburban ls    Unknown        clean  Unknown  Unknown  ...   
5           town country  excellent        clean      fwd  Unknown  ...   
6        outlander sport  excellent        clean      fwd  Unknown  ...   

  cylinders_Unknown cylinders_other fuel_diesel  fuel_electric  fuel_gas  \
0             False           False       False          False      Tr

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Now I need to create training data set and test dataset
# Separate features (X) and target (y)
X = cars_preprocessed.drop("price", axis=1)
y = cars_preprocessed["price"]

# Split the data: 80% training, 20% testing (adjust test_size as needed)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Optionally, print the shapes to verify the split
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

Training set shape: (157738, 77) (157738,)
Testing set shape: (39435, 77) (39435,)
